## WTI Crude Oil Price Prediciton with SVR and LSTM Models

### 1. Introduction

### 1.1 Project Background - Use Case

Although many efforts have been put into green energy development because of the growing concern of climate changes, crude oil is still the most important commodity in the world, because human society heavily depends on energy and synthetic materials produced from fossil fuels. It is very essential to have a better understanding of the potential trend of crude oil prices for policymakers, regulators, investors and producers to manage crude oil production, monitor and evaluate the local and global economies. 

The oil price generally refers to the spot price of a barrel of benchmark crude oil — a reference price for buyers and sellers of crude oil such as West Texas Intermediate (WTI), Brent Crude, Dubai Crude, OPEC Reference Basket, Tapis crude, Bonny Light, Urals oil, Isthmus and Western Canadian Select (WCS). The spot price is the price for a one-time open market transaction for immediate delivery of a specific quantity of product at a specific location, where the commodity is purchased "on the spot" at current market rates. The price of a barrel of oil is different as a result of its grade—determined by factors such as its specific gravity or American Petroleum Institute (API) gravity and its sulphur content—and its location—for example, its proximity to tidewater and refineries. 

The crude oil prices are typically quoted at a particular location. In this project, machine learning (SVR) and deep learning (LSTM) models are developed for an Oil & Gas Commission in North America to predict the WTI spot price based on WTI historical oil price. The price of WTI is a crude stream produced in Texas and southern Oklahoma, which serves as a reference or "marker" for pricing a number of other crude streams and which is traded in the domestic spot market at Cushing, Oklahoma. 

*Sources of Reference: Wikipedia

### 1. 2 Install and Import Python Libraries for Data Retrieval, Analysis and Modeling


In [ ]:
!pip install EIA_python
!pip install julian
!pip install statsmodels
!pip install texttable

import julian
import pandas as pd
import numpy as np
import scipy as sp
import tensorflow as tf
import keras
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
import sklearn.linear_model
import sklearn.metrics
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing import sequence
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Input
from keras.models import Model
import h5py

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.dates as mdates

### 2. Dataset Retrieval, Data Assessment and Exploration

#### 2.1 Retrieve WTI Oil Price Dataset from EIA Database

In [ ]:
# use eia api and api key to retrieve oil price data by series id and create a dataframe
import eia
import os

api_key = "xxxxxxxxxxxxxxxxxxx"
api = eia.API(api_key)

### Retrieve Data By Series ID ###
series_search = api.data_by_series(series = 'PET.RWTC.D')

for key,value in series_search.items():
    print(key,value)

wti = pd.DataFrame(series_search)

#### 2.2 Assess Data Quality of Oil Price Dataset

In [ ]:
# check the starting date of historical data
wti.head()

In [ ]:
# check the end date of dataset
wti.tail()

In [ ]:
# name index column as 'Date'
wti.index.name = 'Date'
wti

In [ ]:
# rename price column as 'WTI Oil Price (USD/Barrel)'
wti = wti.rename(columns = {'Cushing, OK WTI Spot Price FOB, Daily (Dollars per Barrel)':'WTI Oil Price (USD/Barrel)'})
wti

In [ ]:
# reset dateframe index and keep 'Date' column
wti = wti.reset_index(drop=False)
wti

In [ ]:
# set 'Date' format to year-month-day format
from datetime import datetime
wti['Date'] =pd.to_datetime(wti['Date'].str[:-3], format='%Y %m%d')
wti

In [ ]:
# set 'Date' column as index again
wti_date_index = wti.set_index(['Date'])
wti_date_index

In [ ]:
# check summary statistics of wti dataframe
wti.describe()

In [ ]:
# check general information of dataframe: null values, dtypes, count
wti.info()

In [ ]:
# check whether there are missing values by columns
print('Columns: Number of missing values')
print(wti_date_index.isnull().sum(axis=0))

#### 2.3 Data Exploration and Visulization of Oil Price Dataset

In [ ]:
# for a time-series model, data is explored by visulizing the price against date and days
plt.figure(figsize=(8,5))
plt.plot(wti_date_index, color = 'blue', label = 'WTI Historical Crude Oil Prices')
plt.title('WTI Historical Crude Oil Prices')
plt.ylabel('Price (USD/Barrel)')
plt.xlabel('Date')
plt.legend()
plt.show()

In [ ]:
wti_plot = wti.iloc[:,1:2].values.astype(float)

plt.figure(figsize=(8,5))
plt.plot(wti_plot, color = 'blue', label = 'Historical Crude Oil Prices')
plt.title('WTI Historical Crude Oil Prices')
plt.xlabel('Time (Days)')
plt.ylabel('Prices(USD/Barrel)')
plt.legend()
plt.show()

### 3. Predict WTI Crude Oil Price with Support Vector Regression (SVR)

#### 3.1 Prepare Dataset for SVR Modeling

In [ ]:
# Convert date to julian date and add a new column
wti_date_index['Julian'] = wti_date_index.index.to_julian_date()
wti_julian = wti_date_index
wti_julian.head()

In [ ]:
wti_julian_training = wti_julian[:-800]
wti_julian_training.head()

In [ ]:
wti_julian_test = wti_julian[-800:]
wti_julian_test.tail()

In [ ]:
X_training_svr = wti_julian_training["Julian"].to_frame()
y_training_svr = wti_julian_training["WTI Oil Price (USD/Barrel)"].to_frame()

X_test_svr = wti_julian_test["Julian"].to_frame()
y_test_svr = wti_julian_test["WTI Oil Price (USD/Barrel)"].to_frame()

#### 3.2 SVR Modeling and Testset Visulization

In [ ]:
# create model
from sklearn.svm import SVR

oil_svr = SVR(kernel='rbf', C=1e3, gamma=0.001, epsilon=.1).fit(X_training_svr, y_training_svr) 

In [ ]:
# get predicted results
y_pred_svr = list(map(lambda x: float(x), oil_svr.predict(X_test_svr)))

In [ ]:
# plot test dataset
plt.figure(figsize=(8,5))
plt.plot(X_test_svr, y_test_svr, color = 'blue', label = 'Actual Crude Oil Prices')
plt.plot(X_test_svr, y_pred_svr, color = 'red', label = 'Predicted Crude Oil Prices')
plt.title('WTI Crude Oil Prices Prediction with SVR')
plt.xlabel('Julian Date')
plt.ylabel('Crude Oil Prices')
plt.legend()
plt.show()

 #### 3.3 SVR Model Performance Evaluation

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
print("Mean absolute error: %.2f" % mean_absolute_error(y_pred_svr, y_test_svr))

In [ ]:
print("Mean_squared_error: %.2f" % mean_squared_error(y_pred_svr, y_test_svr))

In [ ]:
from sklearn.metrics import r2_score
print("R2-score: %.2f" % r2_score(y_pred_svr, y_test_svr) )

### 4 Predict WTI Crude Oil Price with LSTM Using Sequential API

#### 4.1 Preprocess Data for LSTM Modeling with Sequential API

In [ ]:
# split dataset
sapi_training= wti[:-800]
sapi_training = sapi_training.drop(['Date'], axis = 1)

sapi_test = wti[-800:]
sapi_test = sapi_test.drop(['Date'], axis = 1)

In [ ]:
sapi_training.head()

In [ ]:
sapi_training.tail()

In [ ]:
sapi_test.head()

In [ ]:
sapi_test.tail()

In [ ]:
# feature scaling
scaler = MinMaxScaler()
sapi_training_scaled = scaler.fit_transform(sapi_training)
sapi_training_scaled

In [ ]:
# check training set shape
sapi_training_scaled.shape[0]

In [ ]:
# construct training dataset with 30 days look_back
look_back = 30 # or time steps? 
X_train_sapi = []
y_train_sapi= []

for i in range(look_back, sapi_training_scaled.shape[0]):
    X_train_sapi.append(sapi_training_scaled[i-look_back:i])
    y_train_sapi.append(sapi_training_scaled[i,0])
    
X_train_sapi, y_train_sapi =np.array(X_train_sapi), np.array(y_train_sapi)

In [ ]:
# check the ship of training dataset
X_train_sapi.shape, y_train_sapi.shape

#### 4.2 Build LSTM Model with Sequential API

In [ ]:
# define model using sequential API. Influence of layers on prediction was investigated, but wasn't included in this notebook

regressor = Sequential()

regressor.add(LSTM(units = 50, activation = 'relu', return_sequences = True, input_shape = (X_train_sapi.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
regressor.add(Dropout(0.4))

regressor.add(LSTM(units = 80, activation = 'relu'))
regressor.add(Dropout(0.5))

regressor.add(Dense(units = 1))

# initialising the LSTM model with mse loss function
# adam is fast starting off and then gets slower and more precise
# mse -> mean squared error loss function
regressor.compile(optimizer='adam', loss='mse')

# check model summary
regressor.summary()

In [ ]:
# fit model by inverstigating various values of epochs, batch_size
regressor.fit(X_train_sapi, y_train_sapi, epochs = 30, batch_size = 32)

#### 4.3 Test LSTM Sequential Model

In [ ]:
# check the last 30 days of training dataset
sapi_training.tail(30)

In [ ]:
# get the last 30 days of training dataset
past_30_days = sapi_training.tail(30)

In [ ]:
# append last 30 days of training dataset to test dataset
df_test_sapi = past_30_days.append(sapi_test, ignore_index = True)
df_test_sapi

In [ ]:
# feature scaling of test dataset
inputs_sapi = scaler.transform(df_test_sapi)
inputs_sapi

In [ ]:
# construct test dataset
X_test_sapi = []
y_test_sapi = []

for i in range (look_back, inputs_sapi.shape[0]):
    X_test_sapi.append(inputs_sapi[i-look_back:i])
    y_test_sapi.append(inputs_sapi[i, 0])
    
X_test_sapi, y_test_sapi =np.array(X_test_sapi), np.array(y_test_sapi)
X_test_sapi.shape, y_test_sapi.shape

In [ ]:
# predicted values of test dataset
y_pred_sapi = regressor.predict(X_test_sapi)
y_pred_sapi

In [ ]:
# scaler value
scaler.scale_

In [ ]:
# inversed scaler
scale = 1/0.00740412
scale

In [ ]:
# predicted oil pirce values and oil pirce of test dataset
y_pred_sapi = y_pred_sapi*scale
y_test_sapi = y_test_sapi*scale

In [ ]:
y_pred_sapi

In [ ]:
y_test_sapi

In [ ]:
# visulizing the test data
# plt.plot(wti['WTI'], color = 'red', label = "Real Oil Price")
plt.figure(figsize=(8,5))
plt.plot(y_test_sapi, color = 'blue', label = "Acutal Oil Price")
plt.plot(y_pred_sapi, color = 'red',label = 'Predicted Oil Price')
plt.title('WTI Crude Oil Price Prediction with LSTM Sequential Api')
plt.xlabel('Time (days)')
plt.ylabel('WTI Oil Price')
plt.legend()
plt.show()

#### 4.4 Performance valuation of LSTM Squential Model in Crude Oil Price Prediction

In [ ]:
print("Mean absolute error: %.2f" % mean_absolute_error(y_pred_sapi, y_test_sapi))

In [ ]:
print("Mean_squared_error: %.2f" % mean_squared_error(y_pred_sapi, y_test_sapi))

In [ ]:
print("R2-score: %.2f" % r2_score(y_pred_sapi, y_test_sapi) )

### 5 Predict WTI Crude Oil Price with LSTM Model Using Functional API

#### 5.1 Preprocess Training Dataset for LSTM Model Prediction with Functional API 

In [ ]:
# define the batch size, epochs and timesteps
batch_size = 64
epochs = 160
timesteps = 30

In [ ]:
# split the oil price dataset at a 10% ratio
length = len(wti)
print(length)
length *= 1 - 0.1
print(length)

In [ ]:
# find the remainder of dataset length divided by batch size
7805%64.0

In [ ]:
# find the maximum length with a remainder 0 after being divided by batch size
7744%64.0

In [ ]:
# define a function to get the length of training dataset
def get_train_length(dataset, batch_size, test_percent):
    # substract test_percent to be excluded from training, reserved for testset
    length = len(dataset)
    length *= 1 - test_percent
    train_length_values = []
    for x in range(int(length) - 100,int(length)): 
        modulo=x%batch_size
        if (modulo == 0):
            train_length_values.append(x)
            print(x)
    return (max(train_length_values))

length = get_train_length(wti, batch_size, 0.1)
print(length)

In [ ]:
#Adding timesteps * 2
upper_train = length + timesteps*2
wti_train = wti[0:upper_train]
training_fapi = wti_train.iloc[:,1:2].values
training_fapi.shape

In [ ]:
# Feature Scaling. Scale between 0 and 1. The weights are esier to find.
scaler = MinMaxScaler()
training_fapi_scaled = scaler.fit_transform(np.float64(training_fapi))
training_fapi_scaled.shape

In [ ]:
# construct a training set for LSTM Model using Functional API

X_train_fapi = []
y_train_fapi = []

# Creating a data structure with n timesteps
print(length + timesteps)
for i in range(timesteps, length + timesteps): 
    X_train_fapi.append(training_fapi_scaled[i-timesteps:i,0])
    y_train_fapi.append(training_fapi_scaled[i:i+timesteps,0])

print(len(X_train_fapi))
print(len(y_train_fapi))

#create X_train matrix
#30 items per array (timestep) 
print(X_train_fapi[0:2])
print(np.array(X_train_fapi).shape)

#create Y_train matrix
#30 items per array (timestep) 
print(y_train_fapi[0:2])
print(np.array(y_train_fapi).shape)

In [ ]:
# reshaping training dataset
X_train_fapi, y_train_fapi = np.array(X_train_fapi), np.array(y_train_fapi)
X_train_fapi = np.reshape(X_train_fapi, (X_train_fapi.shape[0], X_train_fapi.shape[1], 1))
y_train_fapi = np.reshape(y_train_fapi, (y_train_fapi.shape[0], y_train_fapi.shape[1], 1))
print(X_train_fapi.shape)
print(y_train_fapi.shape)

#### 5.2 Design LSTM Model Using Functional API

In [ ]:
# initialising the LSTM Model with MSE Loss Function

inputs_1_mse = Input(batch_shape=(batch_size,timesteps,1))
lstm_1_mse = LSTM(10, stateful=True, return_sequences=True)(inputs_1_mse)
lstm_2_mse = LSTM(10, stateful=True, return_sequences=True)(lstm_1_mse)

output_1_mse = Dense(units = 1)(lstm_2_mse)

regressor_mse = Model(inputs=inputs_1_mse, outputs = output_1_mse)

#mse -> mean squared error as loss function
regressor_mse.compile(optimizer='adam', loss = 'mse')
regressor_mse.summary()

In [ ]:
# 1st LSTM layer paramters
parameters = 4 * 10 * (1 + 10 + 1)
print(parameters)

In [ ]:
# 2nd LSTM layer parameters
parameters = 4 * 10 * (10 + 10 + 1)
print(parameters)

In [ ]:
for i in range(epochs):
    print("Epoch: " + str(i))
    # run through all data but the cell, hidden state are used for the next batch.
    # influence of the number of epochs were investigated.
    regressor_mse.fit(X_train_fapi, y_train_fapi, shuffle=False, epochs = 1, batch_size = batch_size)
    # resets only the states but the weights, cell and hidden are kept.
    regressor_mse.reset_states()

In [ ]:
#save model
regressor_mse.save("my_opmodel.h5")
print("Saved model to local")

In [ ]:
#load model
regressor_mse = load_model("my_opmodel.h5")

#### 5.3 Construct Test Dataset

In [ ]:
# define a function to get the length of training dataset
def get_test_length(dataset, batch_size):
    
    test_length_values = []
    for x in range(len(dataset) - 200, len(dataset) - timesteps*2): 
        modulo=(x-upper_train)%batch_size
        if (modulo == 0):
            test_length_values.append(x)
            print(x)
    return (max(test_length_values))

In [ ]:
test_length = get_test_length(wti, batch_size)
print(test_length)
upper_test = test_length + timesteps*2
testset_length = test_length - upper_train
print(testset_length)

In [ ]:
print(upper_train, upper_test, len(wti))

In [ ]:
#subsetting
wti_test_fapi = wti[upper_train:upper_test] 
test_fapi = wti_test_fapi.iloc[:,1:2].values

#scaling
scaled_real_bcg_values_test = scaler.fit_transform(np.float64(test_fapi))

#creating input data
X_test_fapi = []
for i in range(timesteps, testset_length + timesteps):
    X_test_fapi.append(scaled_real_bcg_values_test[i-timesteps:i, 0])
X_test_fapi = np.array(X_test_fapi)


#reshaping
X_test_fapi = np.reshape(X_test_fapi, (X_test_fapi.shape[0], X_test_fapi.shape[1], 1))

In [ ]:
X_test_fapi.shape

#### 5.4 Predict and Visualize of the Testset WTI Crude Oil Price

In [ ]:
# prediction
predicted_bcg_values_test_mse = regressor_mse.predict(X_test_fapi, batch_size=batch_size)
regressor_mse.reset_states()

print(predicted_bcg_values_test_mse.shape)

# reshaping
predicted_bcg_values_test_mse = np.reshape(predicted_bcg_values_test_mse, 
                                       (predicted_bcg_values_test_mse.shape[0], 
                                        predicted_bcg_values_test_mse.shape[1]))
print(predicted_bcg_values_test_mse.shape)

# inverse transform
predicted_bcg_values_test_mse = scaler.inverse_transform(predicted_bcg_values_test_mse)

# creating y_test data
y_pred_fapi = []

for j in range(0, testset_length - timesteps):
    y_pred_fapi = np.append(y_pred_fapi, predicted_bcg_values_test_mse[j, timesteps-1])

# reshaping
y_pred_fapi = np.reshape(y_pred_fapi, (y_pred_fapi.shape[0], 1))

print(y_pred_fapi.shape)

In [ ]:
# visualising the results
plt.figure(figsize=(8,5))
plt.plot(test_fapi[timesteps:len(y_pred_fapi)].astype(float), color = 'blue', label = 'Actual Crude Oil Prices')
plt.plot(y_pred_fapi[0:len(y_pred_fapi) - timesteps], color = 'red', label = 'Predicted Crude Oil Prices')
plt.title('WTI Crude Oil Prices Prediction with LSTM Functional Api')
plt.xlabel('Time (days)')
plt.ylabel('Crude Oil Prices')
plt.legend()
plt.show()

#### 5.5 Evaluate the Performance of LSTM Model in Predicting the Oil Price Using Functional API

In [ ]:
print("Mean absolute error: %.2f" % 
      mean_absolute_error(test_fapi[timesteps:len(y_pred_fapi)], y_pred_fapi[0:len(y_pred_fapi) - timesteps]))

In [ ]:
print("Mean_squared_error: %.2f" % 
      mean_squared_error(test_fapi[timesteps:len(y_pred_fapi)], y_pred_fapi[0:len(y_pred_fapi) - timesteps]))

In [ ]:
print("R2-score: %.2f" % 
      r2_score(test_fapi[timesteps:len(y_pred_fapi)], y_pred_fapi[0:len(y_pred_fapi) - timesteps]) )